In [6]:
import urllib.request, urllib.parse, urllib.error
import json
import ssl
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

# Parsing html in Python with BeautifulSoup and pandas
I used function HTMLTableParser from this resource: http://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/ to parse tables. And some of my own compilations.

In [7]:
class HTMLTableParser:
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table,self.parse_html_table(table))\
            for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):   
# Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
        # Set the number of columns for our table
                    n_columns = len(td_tags)
        
# ------Handle column names if we find them----------------------------------------
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
#----------------------------------------------------------------------------------         

        df = pd.DataFrame(columns = columns,index= range(0,n_rows))
        row_marker = 0     
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                 pass
        return df

In [8]:
def Parsing (url_i, url_w):
    
#Getting Offenders information_1
    hp = HTMLTableParser()     # table-parsing func
    try:
        Inf1 = hp.parse_url(url_i)[0][1]      # Grabbing the table from the tuple      
        for i in range(1,7):                 # The table didn't parsed ideally so it required some adjustments
            Inf1[2][i]=Inf1[1][i]
            Inf1[1][i]=Inf1[0][i]
        Inf1=Inf1.drop(0,axis=1)
        Inf1=Inf1.iloc[:17,:]
        Inf1.columns=['Parameter','Information']
    except:
        Inf1=pd.DataFrame(columns=['Parameter','Information'],index=range(17))
    while len(Inf1)<=16:
        Inf1=Inf1.append({'Parameter': 'New'}, ignore_index=True)
        
#Getting Offenders information_2
    uh = urllib.request.urlopen(url_i, context=ctx).read()     
    soup=BeautifulSoup(uh,"html.parser")
    D=dict()
    tags=soup('p')
    for tag in tags:
        try:
            D[tag.contents[0].get_text()]=re.sub(r"\r\n", "", tag.contents[2]).lstrip()
        except:
            try:
                for i in tag.find_all('span'):
                    D[i.get_text()]=re.sub(r"\r\n", "", tag.contents[3]).lstrip()
            except: a=0
    if D!={}:
        Inf2=pd.DataFrame.from_dict(D,orient='index',columns=['Information']).reset_index().rename(columns={'index':'Parameter'})
        while len(Inf2)<4:
            Inf2=Inf2.append({'Parameter': 'New'}, ignore_index=True)
    else:
        Inf2=pd.DataFrame(columns=['Parameter','Information'],index=range(5))
        
# Getting last words   
    uh = urllib.request.urlopen(url_w, context=ctx).read()
    #data = uh.read().decode()
    soup=BeautifulSoup(uh,"html.parser")
    lst=list()
    for row in soup('div'):
        for column in row.find_all('p'):
            lst.append(column.get_text().strip())
    Inf3=pd.DataFrame(columns=['Parameter','Information'], index=range(0,2))
    if 'Last Statement:' in lst:
        Inf3.iat[1,1]=lst[lst.index('Last Statement:')+1]   
    if 'Date of Execution:' in lst:
        Inf3.iat[0,1]=lst[lst.index('Date of Execution:')+1]
    Inf3['Parameter']=['Date of Execution','Last Statement']
    
    Inf=pd.concat([Inf1,Inf2,Inf3], ignore_index=True)
    Inf.columns=Inf.iloc[0]

    return Inf

## 1.Main page

In [9]:
#Ignore SSL ssertificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url = 'https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html'
uh = urllib.request.urlopen(url, context=ctx).read()
soup=BeautifulSoup(uh,"html.parser")
lst=list()
for t in soup.find_all('table'):
    for row in t.find_all('tr'):
        for tag in row.find_all('td'):
            for a in tag.find_all('a'):
                if a.get('href',None).startswith('/death_row/'):
                    p=a.get('href',None)[11:]
                else:
                    p=a.get('href',None)
                lst.append('https://www.tdcj.texas.gov/death_row/'+p)

## 2. Everything else 

In [10]:
df=Parsing(lst[0],lst[1])
for i in range(2,len(lst)):
    if i%2==0:
        url_i=lst[i]
    else:
        url_w=lst[i]
        print('url_i: ',url_i)
        print('url_w: ',url_w)
        Table=Parsing(url_i,url_w)       
        df=pd.concat([df,pd.DataFrame(Table.iloc[:,1])], axis=1)
df

url_i:  https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohn.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/gardnerjohnlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/runnelstravis.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/runnelstravislast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/halljusten.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/halljustenlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/sparksrobert.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/sparksrobertlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthony.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/solizmarkanthonylast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbilly.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/crutsingerbillylast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarry.html
url_w:  https://www.tdcj.texas.gov/death

url_i:  https://www.tdcj.texas.gov/death_row/dr_info/jenningsrobert.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/jenningsrobertlast.html


url_i:  https://www.tdcj.texas.gov/death_row/dr_info/brazielalvin.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/brazielalvinlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/garciajoseph.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/garciajosephlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/_ramos.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/ramosrobertlast.html


url_i:  https://www.tdcj.texas.gov/death_row/dr_info/ackerdaniel.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/ackerdaniellast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/clarktroy.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/clarktroylast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/youngchristopher.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/youngchristopherlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/bibledanny.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/bibledannylast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/castillojuan.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/castillojuanlast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/davilaerick.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/davilaericklast.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/rodriguezrosendo.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/ro

url_i:  https://www.tdcj.texas.gov/death_row/dr_info/ruizroland.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/ruizrolandolast.html


...

url_i:  https://www.tdcj.texas.gov/death_row/dr_info/skillerndoyle.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/skillerndoylelast.html


url_i:  https://www.tdcj.texas.gov/death_row/dr_info/barefootthomas.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/barefootthomaslast.html


url_i:  https://www.tdcj.texas.gov/death_row/dr_info/obryanronald.jpg
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/obryanronaldlast.html


url_i:  https://www.tdcj.texas.gov/death_row/dr_info/autryjames.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/no_last_statement.html
url_i:  https://www.tdcj.texas.gov/death_row/dr_info/brookscharlie.html
url_w:  https://www.tdcj.texas.gov/death_row/dr_info/brookscharlielast.html


,Name,"Ochoa, Abel Revill","Gardner, John","Runnels, Travis Trevino","Hall, Justen Grant","Sparks, Robert","Soliz, Mark Anthony","Crutsinger, Billy Jack",Larry Ray Swearingen,"King, John William",...,nan,nan,nan,nan,nan,nan,nan,nan,"Autry, James David","Brooks, Charlie Jr."
0,Name,"Ochoa, Abel Revill","Gardner, John","Runnels, Travis Trevino","Hall, Justen Grant","Sparks, Robert","Soliz, Mark Anthony","Crutsinger, Billy Jack",Larry Ray Swearingen,"King, John William",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Autry, James David","Brooks, Charlie Jr."
1,TDCJ Number,999450,999516,999505,999497,999542,999571,999459,999361,999295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,670,592
2,Date of Birth,01/23/1973,01/03/1956,12/17/1972,06/16/1981,02/13/1974,01/27/1982,10/05/1954,5/21/1971,11/03/1974,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9/27/1954,9/1/1942
3,Date Received,05/15/2003,11/20/2006,11/18/2005,05/11/2005,01/08/2009,03/28/2012,10/08/2003,7/12/2000,02/25/1999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10/10/1980,4/25/1978
4,Age (when Received),30,50,32,23,34,30,49,29,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,35
5,Education Level (Highest Grade Completed),12,12,11,9,8,8,11,11,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,12
6,Date of Offense,08/04/2002,01/23/2005,01/29/2003,10/28/2002,09/15/2007,06/29/2010,04/06/2003,12/8/1998,06/07/1998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4/20/1980,12/14/1976
7,Age (at the time of Offense),29,49,30,21,33,28,48,27,23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,34
8,County,Dallas,Collin,Potter,El Paso,Dallas,Johnson,Tarrant,Montgomery,Jasper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jefferson,Tarrant
9,Race,Hispanic,White,Black,White,Black,Hispanic,White,White,White,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,White,Black


In [11]:
df.to_excel('Texas.xlsx')